In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.model_selection import train_test_split, GridSearchCV

from EDA import get_test, get_train, save_pred

In [4]:
X_train, y_train = get_train()
X_test = get_test()


In [5]:
print(X_train.shape)
# print(X_train.head(1))
print(X_test.shape)
print(X_test.dropna().shape)

(19168, 19)
(6966, 19)
(6966, 19)


In [9]:
# 训练随机森林解决回归问题
def random_forest_result(param_grid):
    estimator = RandomForestRegressor()
    model = GridSearchCV(estimator, param_grid, scoring="neg_root_mean_squared_error")
    # print(estimator.get_params().keys())
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print('Best RandomForestRegressor regressor: {}'.format(model.best_params_))

    return model.predict(X_test)

In [6]:
regressor = RandomForestRegressor()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
print(regressor.feature_importances_)
out_file = "data/result_random_forest_1.csv"
save_pred(y_pred, out_file)


[1.11099384e-01 1.72903670e-01 3.07083624e-01 1.73388233e-01
 1.15772299e-03 1.90753659e-03 1.83548085e-01 1.29332064e-02
 2.81785173e-02 3.26130038e-03 3.08103985e-04 7.47658096e-04
 7.82782260e-04 8.50354415e-05 5.34736754e-04 1.63897114e-04
 5.35032265e-04 3.32465414e-04 1.04900912e-03]


In [5]:
# 训练ada_boost解决回归问题
def ada_boost_result(params):
    base_estimator = DecisionTreeRegressor()
    estimator = AdaBoostRegressor(base_estimator=base_estimator)
    # print(estimator.get_params().keys())
    model = GridSearchCV(estimator, params, scoring="neg_root_mean_squared_error")
    model.fit(X_train, y_train)
    # Store the parameters of the best model
    best_params = model.best_params_

    # Predict class labels of test data on the model with the best found parameters
    y_pred = model.predict(X_test)

    print('Best AdaBoost (with DecisionTreeRegressor) regressor: {}'.format(best_params))
    return y_pred

In [11]:
param_grid = {'n_estimators': [50, 100, 150, 200], 'max_features': ["auto", "sqrt", "log2"]}
y_pred = random_forest_result(param_grid=param_grid)
print(y_pred.shape, y_pred)
out_file = "data/result_random_forest.csv"
save_pred(y_pred, out_file)


Best RandomForestRegressor regressor: {'max_depth': 60, 'max_features': 'log2', 'n_estimators': 100}
(6966,) [1417911.21199955 1702794.         1365523.93333333 ... 3258017.5525641
  651086.         3588206.0297619 ]


In [7]:
params = {'base_estimator__max_depth':[6, 8, 9, 10, 12, 15, 20, 25], "n_estimators": [20, 25, 30, 45, 40], "learning_rate": [ 0.5, 0.7, 0.8, 0.9, 1]}
y_pred_ada = ada_boost_result(params)
print(y_pred_ada.shape, y_pred_ada)
out_file = "data/result_ada_boost.csv"
save_pred(y_pred_ada, out_file)

Best AdaBoost (with DecisionTreeRegressor) regressor: {'base_estimator__max_depth': 9, 'learning_rate': 0.8, 'n_estimators': 25}
(6966,) [1544836.01532567 1720462.55319149 1605266.29213483 ... 3197129.50664137
  748592.43243243 3213127.14285714]
